In [14]:
import pandas as pd
import pyarrow as pa
import yfinance as yf
from datetime import timedelta
from prefect import flow, task

In [15]:
@task(retries=3, cache_expiration=timedelta(30))
def fetch_data(ticker):
    return yf.download(ticker)

/home/wesley/.local/lib/python3.10/site-packages/prefect/tasks.py:356: UserWarning: A task named 'fetch_data' and defined at '/tmp/ipykernel_82288/4176811879.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [16]:
@task
def save_data(stock_df):
    stock_df.to_parquet()

/home/wesley/.local/lib/python3.10/site-packages/prefect/tasks.py:356: UserWarning: A task named 'save_data' and defined at '/tmp/ipykernel_82288/3211018998.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [17]:
@flow
def pipeline(ticker):
    df = fetch_data(ticker)
    save_data(df)

/home/wesley/.local/lib/python3.10/site-packages/prefect/flows.py:357: UserWarning: A flow named 'pipeline' and defined at '/tmp/ipykernel_82288/3500445403.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [18]:
if __name__ == '__main__':
    pipeline(ticker="AAPL")

19:52:51.422 | INFO    | prefect.engine - Created flow run 'big-capybara' for flow 'pipeline'

19:52:51.431 | INFO    | Flow run 'big-capybara' - View at https://app.prefect.cloud/account/a2f892c4-4a38-419d-a9e1-2f1c589b6a49/workspace/29760a3e-d175-4bb2-ba3a-056bd84362e4/flow-runs/flow-run/07e2aa3a-1f1d-4086-a68c-75a38832570a

19:52:52.162 | INFO    | Flow run 'big-capybara' - Created task run 'fetch_data-0' for task 'fetch_data'

19:52:52.175 | INFO    | Flow run 'big-capybara' - Executing 'fetch_data-0' immediately...

[*********************100%%**********************]  1 of 1 completed


...

19:52:54.007 | INFO    | Task run 'fetch_data-0' - Finished in state Completed()

19:52:54.336 | INFO    | Flow run 'big-capybara' - Created task run 'save_data-0' for task 'save_data'

19:52:54.339 | INFO    | Flow run 'big-capybara' - Executing 'save_data-0' immediately...

19:52:55.132 | INFO    | Task run 'save_data-0' - Finished in state Completed()

19:52:55.482 | INFO    | Flow run 'big-capybara' - Finished in state Completed('All states completed.')